In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow import keras
import tensorflow as tf
import os
import time
import numpy as np
import pickle
from sklearn.model_selection import RandomizedSearchCV
!pip install scikeras
import scikeras
from scikeras.wrappers import KerasRegressor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


In [3]:
es = tf.keras.callbacks.EarlyStopping(patience=10,min_delta=1.0,verbose=1)

In [4]:
def get_callback(filename,name,value):
    root_logdir = os.path.join(os.curdir, filename)
    ts = int(time.time())
    filen = str(ts)+'_'+str(name)+'_'+str(value)
    return tf.keras.callbacks.TensorBoard(os.path.join(root_logdir, filen))

In [5]:
def build_model(n_hidden, n_neurons, optimizer, learning_rate, momentum=0):
    model = tf.keras.models.Sequential()
    model.add(keras.layers.Input(shape=13))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    if optimizer == 'sgd': 
        opt = keras.optimizers.SGD(lr=learning_rate,nesterov=False)
    elif optimizer == 'nesterov':
        opt = keras.optimizers.SGD(lr=learning_rate,nesterov=True)
    elif optimizer == 'momentum':
        opt = keras.optimizers.SGD(lr=learning_rate,nesterov=False,momentum=momentum)
    elif optimizer == 'adam':
        opt = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='mse',optimizer=opt,metrics=['mse','mae'])
    return model

In [6]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
learning = [0.000001,0.00001,0.0001]
lr = []
for rate in learning:
    model = build_model(1,25,'sgd',rate)
    history = model.fit(X_train,y_train,epochs=100,validation_split=0.1,callbacks=[get_callback('tb_logs','lr',rate),es],verbose=0)
    result = (rate, history.history['loss'][-1], history.history['mae'][-1])
    lr.append(result)
lr

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 89: early stopping
Epoch 22: early stopping


[(1e-06, 298.61773681640625, 13.905632972717285),
 (1e-05, 337.1399230957031, 14.659549713134766),
 (0.0001, 549.4698486328125, 21.24028205871582)]

In [8]:
hidden = [0,1,2,3]
hl = []
for hid in hidden:
    model = build_model(hid,25,'sgd',1e-05)
    history = model.fit(X_train,y_train,epochs=100,validation_split=0.1,callbacks=[get_callback('tb_logs','hl',hid),es],verbose=0)
    result = (hid, history.history['loss'][-1], history.history['mae'][-1])
    hl.append(result)
hl

Epoch 11: early stopping
Epoch 82: early stopping
Epoch 100: early stopping
Epoch 36: early stopping


[(0, 23378.01953125, 72.62944793701172),
 (1, 404.4757080078125, 16.748228073120117),
 (2, 229.35877990722656, 11.249618530273438),
 (3, 411.98870849609375, 16.886764526367188)]

In [9]:
neurons = [5,25,125]
nn = []
for neuron in neurons:
    model = build_model(1,neuron,'sgd',1e-05)
    history = model.fit(X_train,y_train,epochs=100,validation_split=0.1,callbacks=[get_callback('tb_logs','nn',neuron),es],verbose=0)
    result = (neuron, history.history['loss'][-1], history.history['mae'][-1])
    nn.append(result)
nn

Epoch 46: early stopping
Epoch 67: early stopping


[(5, 288.40496826171875, 13.205202102661133),
 (25, 407.7099609375, 16.8358211517334),
 (125, 288.2408752441406, 13.254331588745117)]

In [10]:
opti = ['sgd','nesterov','momentum','adam']
opt = []
for optimizer in opti:
    model = build_model(1,25,optimizer,1e-05,0.5)
    history = model.fit(X_train,y_train,epochs=100,validation_split=0.1,callbacks=[get_callback('tb_logs','opt',optimizer),es],verbose=0)
    result = (optimizer, history.history['loss'][-1], history.history['mae'][-1])
    opt.append(result)
opt

Epoch 46: early stopping


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[('sgd', 368.9898986816406, 15.68124771118164),
 ('nesterov', 474.77978515625, 18.92207145690918),
 ('momentum', 313.5940856933594, 13.945269584655762),
 ('adam', 5499.271484375, 50.31266784667969)]

In [11]:
moment = [0.1,0.5,0.9]
mom = []
for momentum in moment:
    model = build_model(1,25,'momentum',1e-05,momentum)
    history = model.fit(X_train,y_train,epochs=100,validation_split=0.1,callbacks=[get_callback('tb_logs','mom',momentum),es],verbose=0)
    result = (momentum, history.history['loss'][-1], history.history['mae'][-1])
    mom.append(result)
mom

Epoch 99: early stopping
Epoch 46: early stopping


[(0.1, 290.4100341796875, 13.356572151184082),
 (0.5, 393.7789306640625, 16.28229331970215),
 (0.9, 451.1329650878906, 18.036165237426758)]

In [12]:
with open('lr.pkl','wb') as f:
    pickle.dump(lr,f)
with open('hl.pkl','wb') as f:
    pickle.dump(hl,f)
with open('nn.pkl','wb') as f:
    pickle.dump(nn,f)
with open('opt.pkl','wb') as f:
    pickle.dump(opt,f)
with open('mom.pkl','wb') as f:
    pickle.dump(mom,f)

In [13]:
param_distribs = {
"model__n_hidden": [0,1,2,3],
"model__n_neurons": [5,25,125],
"model__learning_rate": [0.000001,0.00001,0.0001],
"model__optimizer": ['sgd','nesterov', 'momentum','adam'],
"model__momentum": [0.1,0.5,0.9]
}

In [14]:
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [15]:
rnd_search_cv = RandomizedSearchCV(keras_reg,param_distribs,n_iter=30,cv=3,verbose=1)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


8/8 [==============================] - 1s 56ms/step - loss: 510.0782 - mse: 510.0782 - mae: 19.8547 - val_loss: 327.7150 - val_mse: 327.7150 - val_mae: 16.1038
Epoch 2/100
8/8 [==============================] - 0s 13ms/step - loss: 471.9808 - mse: 471.9808 - mae: 18.8708 - val_loss: 329.8615 - val_mse: 329.8615 - val_mae: 16.0789
Epoch 3/100
8/8 [==============================] - 0s 17ms/step - loss: 445.7855 - mse: 445.7855 - mae: 18.0715 - val_loss: 317.0804 - val_mse: 317.0804 - val_mae: 15.4875
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 433.0125 - mse: 433.0125 - mae: 17.6360 - val_loss: 314.4315 - val_mse: 314.4315 - val_mae: 15.5062
Epoch 5/100
8/8 [==============================] - 0s 10ms/step - loss: 418.3333 - mse: 418.3333 - mae: 17.2059 - val_loss: 287.6233 - val_mse: 287.6233 - val_mae: 14.3559
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 407.9185 - mse: 407.9185 - mae: 16.9596 - val_loss: 291.8539 - val_mse: 291.8539 -

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

Epoch 1/100


...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


12/12 [==============================] - 1s 18ms/step - loss: 716.9308 - mse: 716.9308 - mae: 19.5155 - val_loss: 303.4658 - val_mse: 303.4658 - val_mae: 14.9663
Epoch 2/100
12/12 [==============================] - 0s 5ms/step - loss: 350.0299 - mse: 350.0299 - mae: 15.1069 - val_loss: 270.2865 - val_mse: 270.2865 - val_mae: 13.6216
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 316.2127 - mse: 316.2127 - mae: 14.0677 - val_loss: 241.8800 - val_mse: 241.8800 - val_mae: 12.5393
Epoch 4/100
12/12 [==============================] - 0s 7ms/step - loss: 311.5391 - mse: 311.5391 - mae: 13.8835 - val_loss: 236.9286 - val_mse: 236.9286 - val_mae: 12.1854
Epoch 5/100
12/12 [==============================] - 0s 5ms/step - loss: 299.4737 - mse: 299.4737 - mae: 13.3899 - val_loss: 227.6804 - val_mse: 227.6804 - val_mae: 11.6782
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: 292.2304 - mse: 292.2304 - mae: 13.1662 - val_loss: 226.0731 - val_mse: 2

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f92b6878090>], model=<function build_model at 0x7f93278d3680>),
                   n_iter=30,
                   param_distributions={'model__learning_rate': [1e-06, 1e-05,
                                                                 0.0001],
                                        'model__momentum': [0.1, 0.5, 0.9],
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': [5, 25, 125],
                                        'model__optimizer': ['sgd', 'nesterov',
                                                             'momentum',
                                                             'adam']},
                   verbose=1)

In [16]:
rnd_search_cv.best_params_

{'model__optimizer': 'sgd',
 'model__n_neurons': 25,
 'model__n_hidden': 3,
 'model__momentum': 0.9,
 'model__learning_rate': 0.0001}

In [17]:
with open('rnd_search.pkl','wb') as f:
    pickle.dump(rnd_search_cv.best_params_,f)

In [18]:
rnd_search_cv.best_score_

nan